In [1]:
from src.vector_search import VectorSearch
import argparse
import logging
import os
from typing import List, Tuple
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from src.data.db import get_connection, release_connection
from src.config import DATA_DIR

load_dotenv()

2025-04-19 05:08:18.564 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/harshavardhan-patil/Work/Projects/canaria/canaria_ml_thp


/home/harshavardhan-patil/Work/Projects/canaria/canaria_ml_thp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Transitivity Analysis
1. Transitivity measures how consistently your system identifies duplicates across the entire dataset, not just in isolated pairs.
2. Transitivity is a fundamental property of well-behaved similarity measures. In a proper vector space, if A is similar to B and B is similar to C, then A should be similar to C. This makes it mathematically sound.
3. We are measuring how many triplets maintain the transitive property out of all available triplets

We get a transitivity score of 88.39 % at 0.95 threshold. This score suggests that 0.95 strikes a good balance - not too strict (which would yield very high transitivity but miss many duplicates) nor too lenient (which would capture more duplicates but have poor transitivity).

### Drawback
Transitivity, by definition, needs triplets, which means we are missing analysis on jobs that were found only twice.

In [12]:
import networkx as nx
import random

threshold = 0.95
sample_size = None

conn = None
try:
    # Connect to database
    conn = get_connection()
    cursor = conn.cursor()
    
    # Get duplicate pairs
    cursor.execute("""
        SELECT lid1, lid2, similarity_score 
        FROM job_duplicates 
    """)
    
    duplicate_pairs = cursor.fetchall()
    
    # Build a graph of job relationships
    G = nx.Graph()
    
    for lid1, lid2, similarity in duplicate_pairs:
        G.add_edge(lid1, lid2, weight=similarity)
    
    # Find all connected components (clusters of similar jobs)
    clusters = list(nx.connected_components(G))
    valid_clusters = [c for c in clusters if len(c) >= 3]
    
    print(f"Found {len(clusters)} total clusters with {sum(len(c) for c in clusters)} jobs")
    print(f"Of these, {len(valid_clusters)} clusters have 3+ jobs and can be checked for transitivity")
    
    total_triplets = 0
    transitive_triplets = 0
    transitivity_violations = []
    
    # Check transitivity for all valid clusters
    for cluster_idx, cluster in enumerate(valid_clusters):
        jobs = list(cluster)
        
        # Generate all possible triplets
        for i in range(len(jobs)):
            for j in range(i+1, len(jobs)):
                for k in range(j+1, len(jobs)):
                    job_a, job_b, job_c = jobs[i], jobs[j], jobs[k]
                    total_triplets += 1
                    
                    # Check if all three pairs exist in the graph
                    ab_exists = G.has_edge(job_a, job_b)
                    bc_exists = G.has_edge(job_b, job_c)
                    ac_exists = G.has_edge(job_a, job_c)
                    
                    # If all direct connections exist, transitivity holds
                    if ab_exists and bc_exists and ac_exists:
                        transitive_triplets += 1
                    else:
                        # Record violation details (limit to 100 for memory efficiency)
                        if len(transitivity_violations) < 100:
                            ab_sim = G.get_edge_data(job_a, job_b)['weight'] if ab_exists else 0
                            bc_sim = G.get_edge_data(job_b, job_c)['weight'] if bc_exists else 0
                            ac_sim = G.get_edge_data(job_a, job_c)['weight'] if ac_exists else 0
                            
                            transitivity_violations.append({
                                'job_a': job_a,
                                'job_b': job_b,
                                'job_c': job_c,
                                'ab_similarity': ab_sim,
                                'bc_similarity': bc_sim, 
                                'ac_similarity': ac_sim
                            })
        
        # Print progress every 100 clusters
        if cluster_idx % 100 == 0 and cluster_idx > 0:
            print(f"Processed {cluster_idx}/{len(valid_clusters)} clusters...")
    
    # Calculate transitivity score
    transitivity_score = transitive_triplets / max(1, total_triplets)
    
    print(f"Transitivity score: {transitivity_score:.4f}")
    print(f"Transitive triplets: {transitive_triplets}/{total_triplets}")
    
except Exception as e:
    print(f"Error calculating transitivity score: {e}")
    raise
finally:
    if conn:
        release_connection(conn)


Found 5828 total clusters with 14923 jobs
Of these, 1467 clusters have 3+ jobs and can be checked for transitivity
Processed 100/1467 clusters...
Processed 200/1467 clusters...
Processed 300/1467 clusters...
Processed 400/1467 clusters...
Processed 500/1467 clusters...
Processed 600/1467 clusters...
Processed 700/1467 clusters...
Processed 800/1467 clusters...
Processed 900/1467 clusters...
Processed 1000/1467 clusters...
Processed 1100/1467 clusters...
Processed 1200/1467 clusters...
Processed 1300/1467 clusters...
Processed 1400/1467 clusters...
Transitivity score: 0.8839
Transitive triplets: 49308/55786


## Supplementary Empirical Analysis
I also analyzed the least confident (lowest similarity score) pairs identified across different approaches and thresholds. This gives us a better look at *where* our system is failing

### Incorrect Duplicate example 1
1. With location filtering (including NULL)
2. Similarity search on combined string of Job title, Company Name, Job Descriptions
3. Threshold: 0.824

Too permissive! This approach allowed 2 different job titles to be identified as duplicates. Since job description dominated the embedding, difference in job title was ignored.

In [6]:
conn = get_connection()
cursor = conn.cursor()

cursor.execute("""
SELECT jobtitle_normalized, finalzipcode, jobdesc_clean from jobs_processed where lid IN ('a276e9c7623d84146f37f879babfbc98','e6fb2c5c1b9eb7a6ef5b29f6d7b2aa29')
""")

cursor.fetchall()

[('pt service associate',
  '19464',
  "Address: USA-PA-Stowe-180 Upland Square Drive \n Store Code: GC - Store Mgrs (2600291) \n \n At The GIANT Company we're committed to making our stores and facilities better every day for our team members, customers, and communities. It's our secret recipe for success and it guides everything we do. We put our customers first, do what's right, win together, and make ideas happen. As a team member here, that's exactly what you'll help us do. You'll share your ideas, learn new skills, and really make a difference for not only your team, but for your customers and your community. \n \n PRIMARY PURPOSE \n \n The primary purpose of this job is to bag items at checkout, put bags into carts, restock returns, retrieve carts from the parking lot, vacuum front-end, restock coolers, and collect trash. \n At The GIANT Company, we are passionate about building strong families and healthy communities, serving millions of neighbors across Pennsylvania, Maryland,

### Incorrect Duplicate example 2
1. With location filtering (including NULL)
2. Heirarchial Similarity Search
3. Theshold: 0.9

Better but still permissive. The jobs are similar, with the only difference between the locations. Unfortunately the location fields did not capture this difference and our threshold was low enough to allow the slight difference in job titles to be classified as duplicates

In [11]:
conn = get_connection()
cursor = conn.cursor()

cursor.execute("""
SELECT jobtitle_normalized, finalzipcode, jobdesc_clean from jobs_processed where lid IN ('53275908b0edeb804c5227e4991ceb95','348495dc429974d7a39daae8c450907f')
""")

cursor.fetchall()

[('family practice travel md winchester ky 1540937 excellent benefits 2025 01 05',
  '40506',
  '\n          LocumJobsOnline is working with The Inline Group to find a qualified Family Practice MD in Winchester, Kentucky, 40391! About The Position Sterling Health Solutions - Full Time  through Friday Employed New Graduates Welcome to Apply Average Patients seen: 20 daily Call Schedule: None Loan Repayment Sign-On Bonus: $20000 Compensation: - Starting Salary: $215,000 - Bonus Potential: $30,000 Benefits: - Eligible providers may apply for NHSC student loan repayment program HIPSA Score 19- Malpractice Premiums fully covered (Federal Tort Claim Coverage)- Eligible to receive up to $5,250 for maintenance of licensure, certification, memberships, continuing education and educational travel related expenses.- Health, Dental and Vision insurance- Prescription Drug Plan- Employer paid Life Insurance 1 times annual salary- Voluntary Life Insurance, Short term disability insurance and other vo

### Correct Duplicate Identifcation example
1. With location filtering (including NULL)
2. Heirarchial Similarity Search
3. Threshold: 0.95

Identified reworded job as duplicate!

At a relatively high threshold and filtering on job title first allows us to capture the subtle differences (and similarities) between the jobs

In [14]:
conn = get_connection()
cursor = conn.cursor()


cursor.execute("""
SELECT jobtitle_normalized, finalzipcode, jobdesc_clean from jobs_processed where lid IN ('3e0876b777d85da96b9f5dbb726526b0','ed707970eee7ca9cd939faf97140f63f')
""")

cursor.fetchall()

[('icu critical care travel rn marrero la 26641098 excellent benefits 2025 01 05',
  '70073',
  "\n          TravelNurseSource is working with Trustaff to find a qualified ICU/Critical Care RN in Marrero, Louisiana, 70072! About The Position About this job As an ICU RN, you'll provide hands-on care for critically ill patients. Intensive Care nurses must be able to assess, plan, and implement care strategies for patients who may suffer from a variety of complex health conditions, but all of whom need round-the-clock attention. Qualifications 1-2 years recent experience in the job's primary area of care Active license or appropriate accreditation in one of the 50 US states Additional requirements & certifications as may be requested by the specificfacility/unit An enthusiastic, adventurous spirit with a desire to help Covid-19 Vaccine Required 26641098EXPPLAT About Trustaff Let Trustaff find your next adventure. Since 2002, Trustaff has matched skilled healthcare professionals with great

In [16]:
query = "SELECT * FROM job_duplicates"
df = pd.read_sql(query, conn)

/tmp/ipykernel_185278/664752144.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [20]:
df = df.rename(columns={"lid1": "Job ID 1", "lid2": "Job ID 2"})
df

,Job ID 1,Job ID 2,similarity_score,created_at
0,9323099c599e3ffb7d0e2b7874c09cfb,ac2177b2cc692134d430051b0a24a28d,1.000000,2025-04-19 09:14:04.898501
1,6eeeaa63a4589a6d1df907cb287b9c1f,3617fb941aa5068000463f933507caab,1.000000,2025-04-19 09:14:04.898501
2,ece005e17da4eed06e4e7012a1efdaf8,e552ab08b3d5b9b8318faafa98a582aa,1.000000,2025-04-19 09:14:04.898501
3,4e712bc635beea5d98ddeff262096b33,51dd2f15052daebe841808385f6015cc,1.000000,2025-04-19 09:14:04.898501
4,4e712bc635beea5d98ddeff262096b33,e3d173b7bdfff64ea3c1e307b1d53edd,1.000000,2025-04-19 09:14:04.898501
...,...,...,...,...
37671,99e6690bba74e476d915a6ccc2071ca7,b5f8b8a87d463ac315b0bddc0cb27bb2,0.999699,2025-04-19 09:14:04.898501
37672,99e6690bba74e476d915a6ccc2071ca7,6c27d3fc797a74cc3b49b6cdb812413b,0.999699,2025-04-19 09:14:04.898501
37673,6519d0e6ed84d3e5a28c34907714a939,6090dcb2dd8d977b58cd7e5fa9133fb2,1.000000,2025-04-19 09:14:04.898501
37674,829f02f266e7affdd4057800889374b1,d48fa3da276eb1c459f8084859cd6e86,0.993269,2025-04-19 09:14:04.898501


In [21]:
df.drop(columns="created_at").to_csv(DATA_DIR / "final_output.csv", index=False)

In [ ]:
release_connection(conn)